In [1]:

import pandas as pd 

dataB = pd.read_csv("./Data/사전테스트-환경데이터/environmentsB.csv")
dataC = pd.read_csv("./Data/사전테스트-환경데이터/environmentsC.csv")
dataD = pd.read_csv("./Data/사전테스트-환경데이터/environmentsD.csv")
dataE = pd.read_csv("./Data/사전테스트-환경데이터/environmentsE.csv")

dataR = pd.read_excel("./Data/사전테스트-생육데이터.xlsx")


In [2]:

def merge_input_output(saveFilename):
    #컬럼이름변경
    # dataR.rename(columns={'생육주사': '주차'}, inplace=True)

        # 전체 데이터를 사용하여 피벗 테이블을 생성합니다
    # '시설아이디' 유무에 따른 row 갯수 오류 확인하기
    # pivot = pd.pivot_table(dataR, 
    #                         values='조사항목값', 
    #                         index=['시설아이디','생육주사', '조사일자', '표본번호'], 
    #                         columns='조사항목', 
    #                         aggfunc='first')
    # print(pivot)
    # # 인덱스를 리셋합니다
    # pivot = pivot.reset_index()
    # pivot = pivot.rename(columns={'생육주사': '주차','착과수': '개화수'})
    pivot = pd.read_csv(f"./Data/pivot.csv",)
    # pivot.head()
    from datetime import datetime, timedelta
    from sklearn.preprocessing import MinMaxScaler
    ## datetime date, time 분리
    datalist = [dataB, dataC, dataD, dataE]

    ## datetime을 date로 변경 
    for data in datalist:
        data['datetime'] = pd.to_datetime(data['datetime'])
        data['date'] = data['datetime'].dt.date
        data['time'] = data['datetime'].dt.hour
    

    base_dateB = datetime(2023, 10, 6)
    base_dateC = datetime(2023, 9, 22)  
    base_dateD = datetime(2023, 10, 18)  
    base_dateE = datetime(2023, 9, 22)  

    base_weekB = 4
    base_weekC = 1
    base_weekD = 4
    base_weekE = 1

    # 주차 계산 함수
    def calculate_week(date, base_date, base_week):
        base_date_timestamp = pd.Timestamp(base_date)

        # 날짜 차이 계산
        delta_days = (date - base_date_timestamp).dt.days

        # 기준 주차에서 날짜 차이를 주 단위로 변환
        week = base_week + delta_days // 7
        return week


    datesB = pd.to_datetime(dataB['date']) 
    datesC = pd.to_datetime(dataC['date']) 
    datesD = pd.to_datetime(dataD['date']) 
    datesE = pd.to_datetime(dataE['date']) 

    weeksB = calculate_week(datesB, base_dateB, base_weekB)
    weeksC = calculate_week(datesC, base_dateC, base_weekC)
    weeksD = calculate_week(datesD, base_dateD, base_weekD)
    weeksE = calculate_week(datesE, base_dateE, base_weekE)

    dataB['weeks'] = weeksB
    dataC['weeks'] = weeksC
    dataD['weeks'] = weeksD
    dataE['weeks'] = weeksE
    

    # 주차와 input 데이터만 데이터셋으로 만듬
    dataCC = dataC.iloc[: ,[2,3,4,5,6,7,10]]
    dataBB = dataB.iloc[: ,[2,3,4,5,6,7,10]]
    dataDD = dataD.iloc[: ,[2,3,4,5,6,7,10]]
    dataEE = dataE.iloc[: ,[2,3,4,5,6,7,10]]

    dataT = pd.concat([dataBB, dataCC, dataDD, dataEE], ignore_index=True)

        # 주차와 input 데이터만 데이터셋으로 만듬
    dataC2 = dataC.iloc[: ,[0,2,3,4,5,6,7,10]]
    dataB2 = dataB.iloc[: ,[0,2,3,4,5,6,7,10]]
    dataD2 = dataD.iloc[: ,[0,2,3,4,5,6,7,10]]
    dataE2 = dataE.iloc[: ,[0,2,3,4,5,6,7,10]]

    # CO2 컬럼 정규화
    datalist2 = [dataB2, dataC2, dataD2, dataE2]

    for data in datalist2:

        min_x = data['innerCO2'] - data['innerCO2'].min()
        min_max = data['innerCO2'].max() - data['innerCO2'].min()
        
        normalCO2 = min_x / min_max
        data['CO2'] = normalCO2
        

    dataE2
    dataT = pd.concat([dataB2, dataC2, dataD2, dataE2], ignore_index=True)

    dataT.rename(columns={"생육주차":"주차"}, inplace=True)



    
    dataT.rename(columns={"주차":"weeks"}, inplace=True)
    pivot.rename(columns={"주차":"weeks"}, inplace=True)     
    # dataT에서 7주차 D농가 데이터 삭제
    dataT = dataT[~((dataT['weeks'] == 7) & (dataT['farm'] == 'D농가'))]

    # pivot에서 7주차 D농가 데이터 삭제
    pivot = pivot[~((pivot['weeks'] == 7) & (pivot['시설아이디'] == 'D농가'))]
    pivot = pivot.iloc[:,1:]   

    dataT[dataT['farm'] == 'B농가'].iloc[:, 1:]

    farms = ['B', 'C', 'D', 'E']
    group_stats = {}

    for farm in farms:
        farm_data = dataT[dataT['farm'] == f'{farm}농가'].iloc[:, 1:]
        group_stats[farm] = farm_data.groupby('weeks').agg(['sum', 'mean', 'min', 'max', 'std','var','median'])
        
        # 열 이름 변경
        group_stats[farm].columns = [f'{col[0]}_{col[1]}' for col in group_stats[farm].columns]
        
        # 인덱스 리셋 및 시설아이디 추가
        group_stats[farm] = group_stats[farm].reset_index()
        group_stats[farm]['시설아이디'] = f'{farm}농가'

    # 모든 농가의 데이터를 하나로 합치기
    result = pd.concat([group_stats[farm] for farm in farms], ignore_index=True)

    # pivot 데이터와 병합 (필요한 경우)
    result = pd.merge(pivot, result, on=['시설아이디', 'weeks'], how='inner')
    print(result.head())
    # 결과를 CSV 파일로 저장
    result.to_csv(f'./Data/{saveFilename}.csv', index=False)


In [3]:
merge_input_output(saveFilename="ML_final")

  시설아이디  weeks      조사일자  표본번호   관부직경    엽병장   엽수    엽장    엽폭  개화수  ...  \
0   B농가      4  20231006     1  12.39  139.0  5.0  79.0  70.0  0.0  ...   
1   B농가      4  20231006     2  12.59  146.0  5.0  78.0  77.0  0.0  ...   
2   B농가      4  20231006     3  13.91  100.0  6.0  73.0  68.0  0.0  ...   
3   B농가      4  20231006     4  10.36  111.0  4.0  83.0  80.0  0.0  ...   
4   B농가      4  20231006     5   9.48  203.0  3.0  94.0  94.0  0.0  ...   

   innerSolar_std  innerSolar_var  innerSolar_median     CO2_sum  CO2_mean  \
0      164.807974    27161.668163               1.67  145.143536  0.885022   
1      164.807974    27161.668163               1.67  145.143536  0.885022   
2      164.807974    27161.668163               1.67  145.143536  0.885022   
3      164.807974    27161.668163               1.67  145.143536  0.885022   
4      164.807974    27161.668163               1.67  145.143536  0.885022   

    CO2_min  CO2_max   CO2_std   CO2_var  CO2_median  
0  0.277643      1.0  0.2

/var/folders/j9/cyqwg1g57ts8dmlvdn5rdyhw0000gn/T/ipykernel_6172/3186797547.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CO2'] = normalCO2
/var/folders/j9/cyqwg1g57ts8dmlvdn5rdyhw0000gn/T/ipykernel_6172/3186797547.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['CO2'] = normalCO2
/var/folders/j9/cyqwg1g57ts8dmlvdn5rdyhw0000gn/T/ipykernel_6172/3186797547.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [4]:
test = pd.read_csv("Data/ML_final.csv")
test.tail()

,시설아이디,weeks,조사일자,표본번호,관부직경,엽병장,엽수,엽장,엽폭,개화수,...,innerSolar_std,innerSolar_var,innerSolar_median,CO2_sum,CO2_mean,CO2_min,CO2_max,CO2_std,CO2_var,CO2_median
1020,E농가,32,20240426,5,19.17,220.0,7.0,78.0,70.0,0.0,...,21.100573,445.234178,81.115,0.687051,0.028627,0.011183,0.052035,0.016433,0.00027,0.021407
1021,E농가,32,20240426,6,29.35,307.0,6.0,97.0,87.0,0.0,...,21.100573,445.234178,81.115,0.687051,0.028627,0.011183,0.052035,0.016433,0.00027,0.021407
1022,E농가,32,20240426,7,29.94,288.0,8.0,83.0,70.0,0.0,...,21.100573,445.234178,81.115,0.687051,0.028627,0.011183,0.052035,0.016433,0.00027,0.021407
1023,E농가,32,20240426,8,23.65,319.0,4.0,95.0,81.0,0.0,...,21.100573,445.234178,81.115,0.687051,0.028627,0.011183,0.052035,0.016433,0.00027,0.021407
1024,E농가,32,20240426,9,27.90,292.0,8.0,80.0,71.0,0.0,...,21.100573,445.234178,81.115,0.687051,0.028627,0.011183,0.052035,0.016433,0.00027,0.021407
